In [1]:
from mainnet_launch.solver_diagnostics.fetch_rebalance_events import fetch_rebalance_events_df

from mainnet_launch.constants import AUTO_LRT

clean_rebalance_df = fetch_rebalance_events_df(AUTO_LRT).reset_index()
clean_rebalance_df

2024-12-12 10:26:04.565 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-12-12 10:26:04.568 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-12-12 10:26:04.569 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-12-12 10:26:04.572 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-12-12 10:26:04.573 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-12-12 10:26:04.575 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-12-12 10:26:04.577 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-12 10:26:04.738 
  command:

    streamlit run /home/park

,timestamp,block,break_even_days,swapCost,swapCostIdle,swapCostChurn,apr_delta,out_compositeReturn,in_compositeReturn,predicted_increase_after_swap_cost,...,moveName,event,gas_price,gas_used,gasCostInETH,flash_borrower_address,before_rebalance_eth_value_of_solver,after_rebalance_eth_value_of_solver,solver_profit,autopool_eth_addr
0,2024-10-05 18:30:59+00:00,20901142,NaN,0.000000,0.0,0.000000,-6.566106,6.566106,0.000000,NaN,...,weETH/ezETH/rswETH (balancer) -> autoLRT (toke...,RebalanceToIdle,14622574084,4322345,0.063204,0x067cB94576aBBDB3527055f3c8CaAAc9668364AD,0.000000,0.000000,0.000000,0xE800e3760FC20aA98c5df6A9816147f190455AF3
1,2024-10-29 21:30:59+00:00,21073995,NaN,0.000000,0.0,0.000000,-4.575516,4.575516,0.000000,NaN,...,rsETH / WETH (balancer) -> autoLRT (tokemak),RebalanceToIdle,11426582933,2880736,0.032917,0x067cB94576aBBDB3527055f3c8CaAAc9668364AD,0.000000,0.000000,0.000000,0xE800e3760FC20aA98c5df6A9816147f190455AF3
2,2024-11-29 16:36:23+00:00,21294579,NaN,0.000000,0.0,0.000000,-7.623399,7.623399,0.000000,NaN,...,rsETH / ETHx (balancer) -> autoLRT (tokemak),RebalanceToIdle,21034106399,3228366,0.067906,0x067cB94576aBBDB3527055f3c8CaAAc9668364AD,0.000000,0.025308,0.025308,0xE800e3760FC20aA98c5df6A9816147f190455AF3
3,2024-09-17 02:30:23+00:00,20767364,2.415104,0.045264,0.0,0.045264,6.642883,0.000000,6.642883,0.891829,...,autoLRT (tokemak) -> weeth-ng (curve),RebalanceBetweenDestinations,6240393542,3081888,0.019232,0x067cB94576aBBDB3527055f3c8CaAAc9668364AD,0.025308,0.000000,-0.025308,0xE800e3760FC20aA98c5df6A9816147f190455AF3
4,2024-09-17 18:34:35+00:00,20772145,0.139109,0.004478,0.0,0.004478,11.191440,0.000000,11.191440,1.605042,...,autoLRT (tokemak) -> rsETH / WETH (balancer),RebalanceBetweenDestinations,18315609347,3065792,0.056152,0x067cB94576aBBDB3527055f3c8CaAAc9668364AD,0.000000,0.021656,0.021656,0xE800e3760FC20aA98c5df6A9816147f190455AF3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,2024-12-11 12:31:47+00:00,21379280,1.565486,0.006781,0.0,0.006781,5.278261,6.694179,11.972440,0.183820,...,weETH/rETH (balancer) -> weeth-ng (curve),RebalanceBetweenDestinations,18762776297,4846346,0.090931,0x067cB94576aBBDB3527055f3c8CaAAc9668364AD,0.049794,0.052635,0.002840,0xE800e3760FC20aA98c5df6A9816147f190455AF3
107,2024-10-09 19:32:47+00:00,20930151,NaN,NaN,0.0,NaN,4.960171,0.229456,5.189627,NaN,...,weETH/rETH (balancer) -> weETH/rETH (balancer),RebalanceBetweenDestinations,31318316690,15791093,0.494550,0x2D363dcC2c19944698e5203bfb76C22CCCd55111,0.000000,0.000000,0.000000,0xE800e3760FC20aA98c5df6A9816147f190455AF3
108,2024-10-11 02:49:35+00:00,20939503,0.000000,0.000000,0.0,0.000000,5.317930,0.209233,5.527164,7.595364,...,weeth-ng (curve) -> weeth-ng (curve),RebalanceBetweenDestinations,12229918454,6225620,0.076139,0x2D363dcC2c19944698e5203bfb76C22CCCd55111,0.000000,0.000000,0.000000,0xE800e3760FC20aA98c5df6A9816147f190455AF3
109,2024-10-12 02:56:47+00:00,20946688,NaN,NaN,0.0,NaN,5.490449,0.000000,5.490449,NaN,...,rsETH / ETHx (balancer) -> rsETH / ETHx (balan...,RebalanceBetweenDestinations,11086762815,16584701,0.183871,0x2D363dcC2c19944698e5203bfb76C22CCCd55111,0.000000,0.000000,0.000000,0xE800e3760FC20aA98c5df6A9816147f190455AF3


In [ ]:
# # sqllite won't perserve datetimes as utc
# df_loaded['timestamp'] = pd.to_datetime(df_loaded['timestamp'], utc=True)

In [7]:
import sqlite3
import pandas as pd


db_file = "autopool_dashboard.db"


def write_df_to_table(df: pd.DataFrame, table_name: str) -> None:
    with sqlite3.connect(db_file) as conn:
        cursor = conn.cursor()

        cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}'")
        table_exists = cursor.fetchone() is not None

        if not table_exists:
            # Create table if it doesn't exist
            df.to_sql(table_name, conn, index=False, if_exists="replace")
            print(f"Table '{table_name}' created and data inserted.")
        else:
            # Load existing table into a DataFrame
            existing_df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)

            # Find rows in df not in existing_df
            new_rows = df[~df.apply(tuple, axis=1).isin(existing_df.apply(tuple, axis=1))]

            if not new_rows.empty:
                # Append new rows to the table
                new_rows.to_sql(table_name, conn, index=False, if_exists="append")
                print(f"Added {len(new_rows)} new rows to '{table_name}'.")
            else:
                print(f"No new rows to add to '{table_name}'.")

        updated_df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
        if not updated_df.equals(df):
            raise ValueError(
                f"Data inconsistency detected between the DataFrame and the table '{table_name}'. Rolling back."
            )
        else:
            print(f"Data consistency verified for table '{table_name}'.")


def load_table_if_exists(table_name: str, where_clause: str | None) -> pd.DataFrame | None:
    """
    Loads data from the specified table if it exists and applies the given WHERE clause.

    Parameters:
        table_name (str): Name of the table to load.
        where_clause (str): SQL WHERE clause to filter the data. example: f"autopool_eth_addr = '{auotpool.autopool_eth_addr}'"

    Returns:
        pd.DataFrame | None: DataFrame containing the queried data or None if the table doesn't exist.
    """
    with sqlite3.connect(db_file) as conn:
        cursor = conn.cursor()

        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?", (table_name,))
        table_exists = cursor.fetchone() is not None
        if not table_exists:
            return None

        if where_clause is not None:
            query = f"SELECT * FROM {table_name}"
        else:
            query = f"SELECT * FROM {table_name} WHERE {where_clause}"

        df = pd.read_sql_query(query, conn)
        return df

IndentationError: expected an indented block after 'if' statement on line 61 (1642428198.py, line 63)

In [6]:
df_loaded["timestamp"]

0     2024-10-05 18:30:59+00:00
1     2024-10-29 21:30:59+00:00
2     2024-11-29 16:36:23+00:00
3     2024-09-17 02:30:23+00:00
4     2024-09-17 18:34:35+00:00
                 ...           
106   2024-12-11 12:31:47+00:00
107   2024-10-09 19:32:47+00:00
108   2024-10-11 02:49:35+00:00
109   2024-10-12 02:56:47+00:00
110   2024-10-12 13:31:23+00:00
Name: timestamp, Length: 111, dtype: datetime64[ns, UTC]

In [4]:
for og, loaded in zip(clean_rebalance_df.dtypes, df_loaded.dtypes):
    print(og, loaded)

datetime64[ns, UTC] datetime64[ns, UTC]
int64 int64
float64 float64
float64 float64
float64 float64
float64 float64
float64 float64
float64 float64
float64 float64
float64 float64
float64 float64
float64 float64
float64 float64
object object
object object
float64 float64
float64 float64
object object
object object
object object
int64 int64
int64 int64
float64 float64
object object
float64 float64
float64 float64
float64 float64
object object


In [10]:
a = pd.DataFrame()
a["a_row"] = [100, None, 20]


b = pd.DataFrame()
b["a_row"] = [100, None, 20]
a.equals(b)

True